In [3]:
import pandas as pd
from scipy.stats import f_oneway

path = '../Data/dataset_clusterizado.xlsx'

df = pd.read_excel(path)

# get numerical features in the DataFrame
numerical_features = df.select_dtypes(include=['float64', 'int64']).columns

# Supongamos que tienes un DataFrame df con una columna 'cluster' y una columna 'feature'
for feature in numerical_features:
  df_cluster1 = df[df['cluster_kmeans'] == 1][feature]
  df_cluster2 = df[df['cluster_kmeans'] == 2][feature]
  df_cluster3 = df[df['cluster_kmeans'] == 3][feature]

  # Realizar ANOVA
  statistic, pvalue = f_oneway(df_cluster1, df_cluster2, df_cluster3)
  print('Feature:', feature)
  print('Estadístico F:', statistic)
  print('Valor p:', pvalue)
  print('')


Feature: alumnos
Estadístico F: 190.8020669849457
Valor p: 2.701599310910165e-71

Feature: 1p_total_actividades_asignadas
Estadístico F: 304.8384300294419
Valor p: 7.076791554338681e-105

Feature: 1p_total_actividades_evaluativas
Estadístico F: 340.51183423774756
Valor p: 1.973072457728449e-114

Feature: 1p_porcentaje_evidencias_calificadas
Estadístico F: 590.6548770779912
Valor p: 5.239923976135107e-172

Feature: 1p_total_actividades_calificadas
Estadístico F: 907.123143495878
Valor p: 1.460028402725776e-228

Feature: 1p_total_actividades_entregadas
Estadístico F: 848.1866567479909
Valor p: 4.755812495593388e-219

Feature: 1p_porcentaje_evidencias_calificadas_tiempo
Estadístico F: 229.42563717900504
Valor p: 3.08110419142416e-83

Feature: 1p_total_actividades_calificadas_tiempo
Estadístico F: 779.5615650082258
Valor p: 1.9266293310905557e-207

Feature: 1p_porcentaje_aprobados
Estadístico F: 271.326187423186
Valor p: 1.623122853763046e-95

Feature: 2p_total_actividades_asignadas
Estadí

/var/folders/bj/qrmf74gn3zx143knsw2l_yk1f6z2jr/T/ipykernel_63707/1376278210.py:18: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  statistic, pvalue = f_oneway(df_cluster1, df_cluster2, df_cluster3)
